In [1]:
%pip install MetaTrader5
%pip install schedule

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [32]:
from datetime import datetime
import time
import pandas as pd
import numpy as np
import plotly.express as px
import MetaTrader5 as mt

In [33]:
mt.initialize(timezone='UTC') # specify the time zone for MetaTrader 5
symbol="XAUUSD"

In [35]:
login = 5010509247
password ='wtua3eay'
server = 'MetaQuotes-Demo'


mt.login(login,password,server)

True

In [36]:
account_info = mt.account_info()
print(account_info)

AccountInfo(login=5010509247, trade_mode=0, leverage=100, limit_orders=200, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=100000.0, credit=0.0, profit=0.0, equity=100000.0, margin=0.0, margin_free=100000.0, margin_level=0.0, margin_so_call=50.0, margin_so_so=30.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='tyler hunt', server='MetaQuotes-Demo', currency='GBP', company='MetaQuotes Software Corp.')


In [37]:
def calculate_atr(df, period):
    true_range = np.maximum(df['high'] - df['low'], df['high'] - df['close'].shift(1), df['close'].shift(1) - df['low'])
    atr = true_range.rolling(window=period).mean()
    return atr

def atr_trailing_stop(src, atr):
    x_atr_trailing_stop = 0.0
    # n_loss = atr * 1
    result = []
    src=src.values
    atr=atr.values
    for i in range(len(src)):
        n_loss = atr[i]
        if i == 0:
            result.append(x_atr_trailing_stop)
        else:
            if src[i] > src[i-1] and src[i-1] > x_atr_trailing_stop:
                x_atr_trailing_stop = max(x_atr_trailing_stop, src[i] - n_loss)
            elif src[i] < src[i-1] and src[i-1] < x_atr_trailing_stop:
                x_atr_trailing_stop = min(x_atr_trailing_stop, src[i] + n_loss)
            elif src[i] > src[i-1]:
                x_atr_trailing_stop = src[i] - n_loss
            else:
                x_atr_trailing_stop = src[i] + n_loss
            result.append(x_atr_trailing_stop)
    return result

def ema(close_price):
    close_price = close_price.values
    ema = close_price[0]
    ema_values = [ema]
    for i in range(1, len(close_price)):
        ema = ema * (1 - 1/1) + close_price[i] * (1/1)
        ema_values.append(ema)
    return ema_values

def buy_sell_signals(src, xATRTrailingStop, ema):
    above_below =''
    res = []
    last_signal = 0;
    for i in range(len(src)):
        # print(i)
        if i !=0:
            
            if (src[i] > xATRTrailingStop[i] and ema[i] > xATRTrailingStop[i] and ema[i-1]<xATRTrailingStop[i-1]):
                above_below = 'above'
                if last_signal!=1:
                    res.append(1)
                    last_signal = 2
                else:
                    res.append(0)
            elif (src[i] < xATRTrailingStop[i] and ema[i] < xATRTrailingStop[i] and ema[i-1]<xATRTrailingStop[i-1]):
                above_below = 'below'
                if last_signal!=-1:
                    res.append(-1)
                    last_signal = -2
                else:
                    res.append(0)
            else:
                res.append(0)
                last_signal = 0
        if i==0:
            res.append(0)
        
    return res

In [38]:
def getData():
    symbol = 'XAUUSD'
    timeframe = mt.TIMEFRAME_M1
    current_time = int(time.time())
    data = pd.DataFrame(data = mt.copy_rates_from(symbol, timeframe, current_time+2*60*60,20))
    return data

In [39]:
def generateSignal(data):
    
    atr = calculate_atr(data,10)
    atrSl = atr_trailing_stop(data['close'], atr)
    ema1 = ema(data['close'])
    signals = buy_sell_signals(data['close'].values, atrSl, ema1)
    signal = signals[-1]
    return signal
    

In [40]:
RiskRewardRatio =2
percentage = 0.0005

In [41]:
def trade():
    data = getData()
    signal = generateSignal(data)
    closePrice = data['close'].values[-1]
    high = data['high'].values[-1]
    low = data['low'].values[-1]
    
    position = mt.positions_get(symbol="XAUUSD")
    print(position)
    lastTime = -99999999999
    if (len(position)>0):
        lastTime = position[0].time
        position_id=position[0].ticket
        profit = position[0].profit
        print(profit)
        if profit < -5 or profit >1:
            deviation=20
            if position[0].type ==0:
                request={
            "action": mt.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 1.0,
            "type": mt.ORDER_TYPE_SELL,
            "position": position_id,
            "price": mt.symbol_info_tick('XAUUSD').bid,
            "deviation": deviation,
            "magic": 234000,
            "comment": "python script close",
            }
                print("Closing Buy")
            else:
                request={
            "action": mt.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 1.0,
            "type": mt.ORDER_TYPE_BUY,
            "position": position_id,
            "price": mt.symbol_info_tick('XAUUSD').ask,
            "deviation": deviation,
            "magic": 234000,
            "comment": "python script close",
            }
                print("closing sell")
            mt.order_send(request)
    if int(lastTime) - (time.time()+60*60*2)<-60:
        if signal == -1:
            price = mt.symbol_info_tick("XAUUSD").bid
            sl1 =price + 5
            tp1 = price -10
            print(price,sl1,tp1)
            request = {
        "action":mt.TRADE_ACTION_DEAL,
        "symbol":"XAUUSD",
        "volume":1.0,
        "type":mt.ORDER_TYPE_SELL,
        "price": mt.symbol_info_tick("XAUUSD").bid,
        "deviation":20,
        "magic":234000,
        "comment":"",
        "type_time":mt.ORDER_TIME_GTC,
        "type_filling":mt.ORDER_FILLING_IOC,
        }
            print("Sell")
            order = mt.order_send(request)
        
        elif signal == 1:
        
            price = mt.symbol_info_tick("XAUUSD").ask
            sl1 =price -5
            tp1 = price +10
            print(sl1,tp1)
            request = {
        "action":mt.TRADE_ACTION_DEAL,
        "symbol":"XAUUSD",
        "volume":1.0,
        "type":mt.ORDER_TYPE_BUY,
        "price":mt.symbol_info_tick("XAUUSD").ask,
        "deviation":20,
        "magic":234000,
        "comment":"",
        "type_time":mt.ORDER_TIME_GTC,
        "type_filling":mt.ORDER_FILLING_IOC,
        }
            print("Buy")
            order = mt.order_send(request)
           
    

In [42]:
def tradeWithStopLoss():
    data = getData()
    signal = generateSignal(data)
    closePrice = data['close'].values[-1]
    high = data['high'].values[-1]
    low = data['low'].values[-1]
    if signal == -1:
        price = mt.symbol_info_tick("XAUUSD").bid
        sl1 = price*1.001
        tp1 = price - (sl1-price)*3
        print(price,sl1,tp1)
        request = {
        "action":mt.TRADE_ACTION_DEAL,
        "symbol":"XAUUSD",
        "volume":1.0,
        "type":mt.ORDER_TYPE_SELL,
        "price": mt.symbol_info_tick("XAUUSD").bid,
        "sl":sl1,
        "tp":tp1,
        "deviation":20,
        "magic":234000,
        "comment":"",
        "type_time":mt.ORDER_TIME_GTC,
        "type_filling":mt.ORDER_FILLING_IOC,
        }
        print("Sell")
        order = mt.order_send(request)
        
    elif signal == 1:
        price = mt.symbol_info_tick("XAUUSD").ask
        sl1 =price*0.999
        tp1 = price+(price-sl1)*3
        print(sl1,tp1)
        request = {
        "action":mt.TRADE_ACTION_DEAL,
        "symbol":"XAUUSD",
        "volume":1.0,
        "type":mt.ORDER_TYPE_BUY,
        "price":mt.symbol_info_tick("XAUUSD").ask,
        "sl":sl1,
        "tp":tp1,
        "deviation":20,
        "magic":234000,
        "comment":"",
        "type_time":mt.ORDER_TIME_GTC,
        "type_filling":mt.ORDER_FILLING_IOC,
        }
        print("Buy")
        order = mt.order_send(request)
           
    

In [43]:

# CONFIGS
# TICKET = 165856631
MAX_DIST_SL = 0.5  # Max distance between current price and SL, otherwise SL will update
TRAIL_AMOUNT = 0.5  # Amount by how much SL updates
DEFAULT_SL = 0.3  # If position has no SL, set a default SL


# function to trail SL
def trail_sl(ticket):
    # get position based on ticket_id
    position = mt.positions_get(ticket=ticket)

    # check if position exists
    if position:
        position = position[0]
    else:
        print('Position does not exist')
        sys.exit()

    # get position data
    order_type = position.type
    price_current = position.price_current
    price_open = position.price_open
    sl = position.sl
    
    print(order_type,price_open,sl)

    dist_from_sl = abs(round(price_current - sl, 6))
    print(dist_from_sl)
    if dist_from_sl > MAX_DIST_SL:
        # calculating new sl
        if sl != 0.0:
            if order_type == 0:  # 0 stands for BUY
                if price_current > price_open:
                    
                    if sl > price_open:
                        ta = 0.01
                    else:
                        ta = TRAIL_AMOUNT
                    request = {
                    'action': mt.TRADE_ACTION_SLTP,
                    'position': position.ticket,
                    'sl': sl + ta,
                    }

                    result = mt.order_send(request) 
                    return result
                # new_sl = sl + TRAIL_AMOUNT
                # print(new_sl)
            elif order_type == 1:  # 1 stands for SELL
                # print(price_current, price_open)
                if price_current < price_open:
                    if sl < price_open:
                        ta = 0.01
                    else:
                        ta = TRAIL_AMOUNT
                    print(price_current, price_open)
                    request = {
                    'action': mt.TRADE_ACTION_SLTP,
                    'position': position.ticket,
                    'sl': sl - ta,
                    }

                    result = mt.order_send(request)
                    print(result)
                    return result
#                 new_sl = sl - TRAIL_AMOUNT
#                 print(new_sl)

        else:
            # setting default SL if the is no SL on the symbol
            new_sl = price_open - DEFAULT_SL if order_type == 0 else price_open + DEFAULT_SL
            print(new_sl)
        
            request = {
                'action': mt.TRADE_ACTION_SLTP,
                'position': position.ticket,
                'sl': new_sl,
            }

            result = mt.order_send(request)
            print(result)
        
            return result

In [45]:
import schedule
schedule.every().minute.at(":00").do(trade)
while True:
    positions = mt.positions_get(symbol="XAUUSD")
    if (len(positions)==1):
        trail_sl(positions[0].ticket)
    else:
        trade()
    time.sleep(5)

()
()
()
()
()
()
()
()
()
()
()
1844.69 1849.69 1834.69
Sell
()
1844.61 1849.61 1834.61
Sell
()
1844.66 1849.66 1834.66
Sell
()
1844.64 1849.64 1834.64
Sell
()
1844.62 1849.62 1834.62
Sell
()
1844.64 1849.64 1834.64
Sell
()
1844.69 1849.69 1834.69
Sell
()
1844.69 1849.69 1834.69
Sell
()
1844.65 1849.65 1834.65
Sell
()
1844.78 1849.78 1834.78
Sell
()
1844.78 1849.78 1834.78
Sell
()
1844.78 1849.78 1834.78
Sell
()
1839.88 1854.88
Buy
()
1839.92 1854.92
Buy
()
1840.1 1855.1
Buy
()
1840.06 1855.06
Buy
()
1840.03 1855.03
Buy
()
1840.27 1855.27
Buy
()
1840.15 1855.15
Buy
()
1840.22 1855.22
Buy
()
1840.21 1855.21
Buy
()
1840.21 1855.21
Buy
()
1840.21 1855.21
Buy
()
1840.01 1855.01
Buy
()
()
()
()
()
()
()
()
()
()
()
()
()
1844.86 1849.86 1834.86
Sell
()
1840.03 1855.03
Buy
()
1840.07 1855.07
Buy


KeyboardInterrupt: 

In [ ]:
        # BUY ORDER
        request = {
        "action":mt.TRADE_ACTION_DEAL,
        "symbol":"XAUUSD",
        "volume":1.0,
        "type":mt.ORDER_TYPE_BUY,
        "price":mt.symbol_info_tick("XAUUSD").ask,
        "deviation":20,
        "magic":234000,
        "comment":"",
        "type_time":mt.ORDER_TIME_GTC,
        "type_filling":mt.ORDER_FILLING_IOC,
        }
        print("Buy")
        order = mt.order_send(request)
        print(order)

In [53]:
schedule.every().minute.at(":00").do(trade)
while True:
    positions = mt.positions_get(symbol="XAUUSD")
    print(positions)
    time.sleep(1)

()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
(

(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1844.96, swap=0.0, profit=0.83, symbol='XAUUSD', comment='', external_id=''),)
(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1844.96, swap=0.0, profit=0.83, symbol='XAUUSD', comment='', external_id=''),)
(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1844.96, swap=0.0, profit=0.83, symbol='XAUUSD', comment='', external_id=''),)
(TradePosition(ticket=503603752

(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1844.83, swap=0.0, profit=-1.33, symbol='XAUUSD', comment='', external_id=''), TradePosition(ticket=50360376205, time=1676892714, time_msc=1676892714111, time_update=1676892714, time_update_msc=1676892714111, type=1, magic=0, identifier=50360376205, reason=0, volume=0.2, price_open=1844.83, sl=0.0, tp=0.0, price_current=1844.92, swap=0.0, profit=-1.5, symbol='XAUUSD', comment='', external_id=''))
(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1844.84, swap=0.0, profit=-1.16, symbol='XAUUSD', comment='', external_id=''), TradePosition(ticket=50360376205,

(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1844.93, swap=0.0, profit=0.33, symbol='XAUUSD', comment='', external_id=''), TradePosition(ticket=50360376205, time=1676892714, time_msc=1676892714111, time_update=1676892714, time_update_msc=1676892714111, type=1, magic=0, identifier=50360376205, reason=0, volume=0.2, price_open=1844.83, sl=0.0, tp=0.0, price_current=1845.01, swap=0.0, profit=-2.99, symbol='XAUUSD', comment='', external_id=''))
(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1845.01, swap=0.0, profit=1.6600000000000001, symbol='XAUUSD', comment='', external_id=''), TradePosition(ticket

(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1844.92, swap=0.0, profit=0.17, symbol='XAUUSD', comment='', external_id=''),)
(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1844.92, swap=0.0, profit=0.17, symbol='XAUUSD', comment='', external_id=''),)
(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1844.92, swap=0.0, profit=0.17, symbol='XAUUSD', comment='', external_id=''),)
(TradePosition(ticket=503603752

(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1845.02, swap=0.0, profit=1.83, symbol='XAUUSD', comment='', external_id=''), TradePosition(ticket=50360376768, time=1676892746, time_msc=1676892746938, time_update=1676892746, time_update_msc=1676892746938, type=1, magic=0, identifier=50360376768, reason=0, volume=0.2, price_open=1844.92, sl=0.0, tp=0.0, price_current=1845.1, swap=0.0, profit=-2.99, symbol='XAUUSD', comment='', external_id=''), TradePosition(ticket=50360376829, time=1676892752, time_msc=1676892752713, time_update=1676892752, time_update_msc=1676892752713, type=1, magic=0, identifier=50360376829, reason=0, volume=0.2, price_open=1844.99, sl=0.0, tp=0.0, price_current=1845.1, swap=0.0, profit=-1.83, symbol='XAUUSD', comment='', external_id=''))
(TradePosition(ticket=50360375279, t

(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1844.96, swap=0.0, profit=0.83, symbol='XAUUSD', comment='', external_id=''), TradePosition(ticket=50360376768, time=1676892746, time_msc=1676892746938, time_update=1676892746, time_update_msc=1676892746938, type=1, magic=0, identifier=50360376768, reason=0, volume=0.2, price_open=1844.92, sl=0.0, tp=0.0, price_current=1845.05, swap=0.0, profit=-2.16, symbol='XAUUSD', comment='', external_id=''))
(TradePosition(ticket=50360375279, time=1676892665, time_msc=1676892665618, time_update=1676892665, time_update_msc=1676892665618, type=0, magic=0, identifier=50360375279, reason=0, volume=0.2, price_open=1844.91, sl=0.0, tp=0.0, price_current=1844.96, swap=0.0, profit=0.83, symbol='XAUUSD', comment='', external_id=''), TradePosition(ticket=50360376768, 

()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()
()


KeyboardInterrupt: 